### Edgar Moises Hernandez-Gonzalez
#### 24-07-20-26-08-20
#### Clasificacion binaria de imagenes motoras en EEG con CNN-2D
#### Cargar modelo para evaluacion y predecir
##### Caracteristicas = Espectrogramas STFT (3D)
##### Entrada 135x31

In [1]:
from google.colab import drive

In [49]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
import time
from scipy.signal import spectrogram
from sklearn.metrics import cohen_kappa_score, confusion_matrix
from keras.models import load_model

In [4]:
x_train = pd.read_csv("/content/drive/My Drive/BCI-IV-2b/Datos/MI-EEG-B9T.csv", header=None)
x_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2b/Datos/MI-EEG-B9E.csv", header=None)
y_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2b/Datos/etiquetas_test_9.csv", header=None)

In [5]:
print(x_train.shape)
print(x_test.shape)
print(y_test.shape)

(400, 3000)
(320, 3000)
(320, 1)


In [6]:
# esto se podria calcular asi n_clases = len(np.unique(y)), la y puede ser train o test
n_classes = len(np.unique(y_test))

print("n_classes:", n_classes)

n_classes: 2


In [7]:
#unir 3 canales
def unir_espectrogramas_vertical(data):
  fs = 250
  datos = np.zeros((data.shape[0],135,31))
  temporal = np.zeros((135,31))
  for i in range(data.shape[0]): #n muestras
    for j in range(3): #3 canales
      sig = data.iloc[i, j*1000:(j+1)*1000]
      #sig = sig[124:624]
      #espectrograma de STFFT
      f, t, Sxx = spectrogram(sig, fs=250, window='hann', nperseg=250,
                              noverlap=225, nfft=500, scaling='spectrum')
      temporal[j*45:(j+1)*45, :] = Sxx[16:61, :]
    datos[i] = temporal
    if i % 100 == 0: #esto solo es para ver como avanza
      print(i)
  return datos

In [8]:
#llamar a unir_espectrogramas_vertical(data)
inicio = time.time()

x_train = unir_espectrogramas_vertical(x_train)
x_test = unir_espectrogramas_vertical(x_test)

fin = time.time()
print("Tiempo:", fin - inicio)

0
100
200
300
0
100
200
300
Tiempo: 1.8560855388641357


In [9]:
print(x_test.shape)

(320, 135, 31)


In [10]:
#convertir a float
x_test = x_test.astype('float32')

#escalar los valores en un rango de 0 a 1 (normalizar)
x_test /= np.ceil(np.max(x_train))

In [11]:
#convertir de 3D a 4D
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], x_test.shape[2], 1))

print(x_test.shape)

(320, 135, 31, 1)


In [12]:
model = load_model('/content/drive/My Drive/BCI-IV-2b/Modelos/STFT_CNN_09.hdf5')

In [13]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 135, 31, 4)        40        
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 67, 15, 4)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 67, 15, 4)         148       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 33, 7, 4)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 924)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                29600     
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)               

In [45]:
inicio = time.time()

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)

fin = time.time()
print("Tiempo:", fin - inicio)

Tiempo: 0.28950929641723633


In [ ]:
print("loss: %.4f" % test_loss)
print("accuracy: %.4f" % test_acc)

loss: 0.5589
accuracy: 0.8188


In [46]:
probabilidades = model.predict(x_test)
y_pred = np.argmax(probabilidades, 1) #1=fila

Tiempo: 0.30560755729675293


In [ ]:
kappa = cohen_kappa_score(y_test, y_pred)
print("kappa: %.4f " % kappa)

kappa: 0.6375 


In [ ]:
matriz_confusion = confusion_matrix(y_test, y_pred)
print("confusion matrix:\n", matriz_confusion)

confusion matrix:
 [[124  36]
 [ 22 138]]


In [48]:
inicio = time.time()

probabilidades = model.predict(x_test[0].reshape(1,135,31,1))
y_pred = np.argmax(probabilidades, 1) #1=fila

fin = time.time()
print("Tiempo:", fin - inicio)

Tiempo: 0.08356451988220215
